In [1]:
##################################################################################
##### Define all parameters for model tuning
##################################################################################

n_fold = 5
expName = "PSI_Site_DLNN_MergedSeq_autoencoder"
outPath = "Results"
foldName = "folds.pickle"

epochs = 100
batch_size = 32
shuffle = False
seed = None


input_data_folder = "Data\\Aziz"

In [2]:
import os 
from Bio import SeqIO
import pickle
import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, confusion_matrix
from sklearn.metrics import roc_auc_score

import math

In [3]:
# print(tf.test.is_gpu_available(cuda_only=True))
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[0], True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
##################################################################################
##### define all CUSTOM functions
##################################################################################

def one_hot_encode_dna(sequence):
    seq_encoded = np.zeros((len(sequence),4))
    dict_nuc = {
        "A": 0,
        "C": 1,
        "G": 2,
        "T": 3
    }
    i = 0
    for single_character in sequence:
        if(single_character.upper() in dict_nuc.keys()):
            seq_encoded[i][dict_nuc[single_character.upper()]] = 1
            i = i+1
        else:
            raise ValueError('Incorrect character in DNA sequence: '+sequence)
    return seq_encoded

def one_hot_encode_rna(sequence):
    seq_encoded = np.zeros((len(sequence),4))
    dict_nuc = {
        "A": 0,
        "C": 1,
        "G": 2,
        "U": 3
    }
    i = 0
    for single_character in sequence:
        if(single_character.upper() in dict_nuc.keys()):
            seq_encoded[i][dict_nuc[single_character.upper()]] = 1
            i = i+1
        else:
            raise ValueError('Incorrect character in RNA sequence: '+sequence)
    return seq_encoded

def one_hot_encode_rnafold(sequence):
    seq_encoded = np.zeros((len(sequence),3))
    dict_fold = {
        "(": 0,
        ")": 1,
        ".": 2
    }
    i = 0
    for single_character in sequence:
        if(single_character in dict_fold.keys()):
            seq_encoded[i][dict_fold[single_character]] = 1
            i = i+1
        else:
            raise ValueError('Incorrect character in RNAfold: '+sequence)
    return seq_encoded

def one_hot_encode_rna_mergedseq(sequence):
    dict_nuc = {
        "A": 0,
        "C": 1,
        "G": 2,
        "U": 3
    }
    dict_fold = {
        "(": 0,
        ")": 1,
        ".": 2
    }
    list_seq = sequence.strip().split(' ')
    seq_encoded = np.zeros((len(list_seq),12))
    i = 0
    for single_character in list_seq:
        if(single_character[0].upper() in dict_nuc.keys()):
            idx1 = dict_nuc[single_character[0].upper()]+1
        else:
            raise ValueError('Incorrect RNA character in MergedSeq sequence: '+sequence)
        if(single_character[1] in dict_fold.keys()):
            idx2 = dict_fold[single_character[1]]+1
        else:
            raise ValueError('Incorrect RNAfold character in MergedSeq sequence: '+sequence)
        idx = (idx1 * idx2) - 1
        seq_encoded[i][idx] = 1
        i = i+1        
    return seq_encoded

In [5]:
##################################################################################
##### define evaluator functions
##################################################################################

## Build the K-fold from dataset
def build_kfold(features, labels, k=10, shuffle=False, seed=None):
    
    skf = StratifiedKFold(n_splits=k, shuffle=shuffle, random_state=seed)
    kfoldList = []
    for train_index, test_index in skf.split(features, labels):
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        kfoldList.append({
            "X_train": X_train,
            "X_test": X_test,
            "y_train":y_train,
            "y_test":y_test
        })
    return kfoldList

def build_kfold_multifeature(features_1, features_2, labels, k=10, shuffle=False, seed=None):
    
    skf = StratifiedKFold(n_splits=k, shuffle=shuffle, random_state=seed)
    kfoldList = []
    for train_index, test_index in skf.split(features_1, labels):
        X1_train, X1_test = features_1[train_index], features_1[test_index]
        X2_train, X2_test = features_2[train_index], features_2[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        kfoldList.append({
            "X1_train": X1_train,
            "X1_test": X1_test,
            "X2_train": X2_train,
            "X2_test": X2_test,
            "y_train":y_train,
            "y_test":y_test
        })
    return kfoldList

def pred2label(y_pred):
    y_pred = np.round(np.clip(y_pred, 0, 1))
    return y_pred

In [6]:
##################################################################################
##### Function to customize the DLNN architecture with parameters
##################################################################################

def DLNN_AutoEncoder_Classifier(input_shape = (21,12), 
                                output_shape = (21, 4), 
                                latent_dim = 20
                               ):
    
    
    ae_input = tf.keras.layers.Input(shape=input_shape)
    
    ###########################################################################
    ##### Encoder
    ###########################################################################
    
    encoder = tf.keras.models.Sequential()
    encoder.add(tf.keras.layers.Flatten(input_shape = input_shape))
    encoder.add(tf.keras.layers.Dense(100))
    encoder.add(tf.keras.layers.Dense(latent_dim))
    
    ###########################################################################
    ##### Decoder
    ###########################################################################
    
    decoder = tf.keras.models.Sequential()
    decoder.add(tf.keras.layers.Dense(100, input_shape=(latent_dim,)))
    decoder.add(tf.keras.layers.Dense(output_shape[0]*output_shape[1]))
    decoder.add(tf.keras.layers.Reshape(output_shape))
    decoder.add(tf.keras.layers.Activation('softmax'))
    
    ###########################################################################
    ##### Classifier
    ###########################################################################
    
    classifier = tf.keras.models.Sequential()
    classifier.add(tf.keras.layers.Dense(10, input_shape=(latent_dim,)))
    classifier.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    
    ###########################################################################
    ##### Total Network
    ###########################################################################
    
    autoencoder = tf.keras.models.Model(ae_input, [decoder(encoder(ae_input)), classifier(encoder(ae_input))])
    
    autoencoder.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), 
                        loss=['mean_squared_error', 'binary_crossentropy'], 
                        metrics='accuracy')
    
    return autoencoder, encoder, decoder, classifier

In [7]:
ae,_,_,_ = DLNN_AutoEncoder_Classifier()
ae.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 21, 12)]     0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 20)           27320       input_1[0][0]                    
                                                                 input_1[0][0]                    
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 21, 4)        10584       sequential[1][0]                 
__________________________________________________________________________________________________
sequential_2 (Sequential)       (None, 1)            221         sequential[2][0]             

In [8]:
# model, encoder, decoder, classifier = DLNN_AutoEncoder_Classifier()

In [9]:
##################################################################################
##### For each input file, train model and generate different outputs in a structured folder
##################################################################################

## create the evaluation data structure for all iterations
evaluations = {
    "Model" : [],
    "Kernel_Length" : [],
    "Dataset" : [],
    "Fold" : [],
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

for root, dirs, files in os.walk(input_data_folder):
    for file in files:
        
        input_data_file = os.path.join(root, file)
        
        current_dataset_variety = input_data_file.split("\\")[-1].split(".")[0]

        csv_data = pd.read_csv(input_data_file)

        ##################################################################################
        ##### extract data from the current CSV file
        ##################################################################################
        
        csv_data["OHE"] = pd.Series([one_hot_encode_rna(val) for val in csv_data["Sequence"]])
        csv_data["OHE_MergedSeq"] = pd.Series([one_hot_encode_rna_mergedseq(val) for val in csv_data["MergedSeq"]])

        df_positive = csv_data[csv_data['Number'].str.contains("P")]
        df_negative = csv_data[csv_data['Number'].str.contains("N")]

        positive_ohe_mergedseq = np.array(list(df_positive['OHE_MergedSeq']))
        negative_ohe_mergedseq = np.array(list(df_negative['OHE_MergedSeq']))
        
        positive_ohe_seq = np.array(list(df_positive['OHE']))
        negative_ohe_seq = np.array(list(df_negative['OHE']))

        print("\n======================================================================")
        print("\nFile:", input_data_file)
        print("Positive:", positive_ohe_mergedseq.shape[0])
        print("Negative:", negative_ohe_mergedseq.shape[0])

        ##################################################################################
        ##### Generate Folds from dataset, and store to file
        ##################################################################################

        ## create the features and labels datasets for the training
        input_size = positive_ohe_mergedseq[0].shape
        output_size = positive_ohe_mergedseq[0].shape

        labels = np.concatenate((np.ones((df_positive.shape[0], 1), 
                                         dtype=np.float32), 
                                 np.zeros((df_negative.shape[0], 1), 
                                          dtype=np.float32)), 
                                axis=0)
        features_mergedseq = np.concatenate((positive_ohe_mergedseq, 
                                             negative_ohe_mergedseq), 
                                            axis=0)
        
        features_seq = np.concatenate((positive_ohe_seq, 
                                       negative_ohe_seq), 
                                      axis=0)

        folds = build_kfold_multifeature(features_mergedseq, features_seq, labels, 
                                         k=n_fold, shuffle=shuffle, seed=seed)

#         ## Write the k-fold dataset to file
#         foldPath = os.path.join(outPath, expName, current_dataset_variety, "{}fold".format(n_fold))
#         if(not os.path.isdir(foldPath)):
#             os.makedirs(foldPath)
#         pickle.dump(folds, open(os.path.join(foldPath, foldName), "wb"))

#         ## Create and set directory to save model
#         modelPath = os.path.join(outPath, expName, current_dataset_variety, "{}fold".format(n_fold), "models")
#         if(not os.path.isdir(modelPath)):
#             os.makedirs(modelPath)

        ##################################################################################
        ##### TRAIN and PREDICT for every Fold, using models
        ##################################################################################

        # fold counter
        i = 0

        for fold in folds:

            print("\nTrain/Test model "+current_dataset_variety+" on Fold #"+str(i)+".")
            
            model, encoder, decoder, classifier = DLNN_AutoEncoder_Classifier(input_size, output_size)

            model.fit(x = fold["X1_train"], y = [fold["X1_train"], fold["y_train"]], 
                      batch_size = batch_size, epochs = epochs, 
                      verbose = 1, validation_split=0.2)

            ##################################################################################
            ##### Prediction and metrics for TRAIN dataset
            ##################################################################################

            y_seq, y_pred = model.predict(fold["X1_train"])
            label_pred = pred2label(y_pred)
            # Compute precision, recall, sensitivity, specifity, mcc
            acc = accuracy_score(fold["y_train"], label_pred)
            prec = precision_score(fold["y_train"],label_pred)

            conf = confusion_matrix(fold["y_train"], label_pred)
            if(conf[0][0]+conf[1][0]):
                sens = float(conf[0][0])/float(conf[0][0]+conf[1][0])
            else:
                sens = 0.0
            if(conf[1][1]+conf[0][1]):
                spec = float(conf[1][1])/float(conf[1][1]+conf[0][1])
            else:
                spec = 0.0
            if((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0])):
                mcc = (float(conf[0][0])*float(conf[1][1]) - float(conf[1][0])*float(conf[0][1]))/math.sqrt((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0]))
            else:
                mcc= 0.0
            fpr, tpr, thresholds = roc_curve(fold["y_train"], y_pred)
            auc = roc_auc_score(fold["y_train"], y_pred)
            
            y_seq_ohe = np.eye(12)[np.argmax(y_seq, axis = -1)]
            

            evaluations["Model"].append(current_dataset_variety)
            evaluations["Kernel_Length"].append(None)
            evaluations["Dataset"].append(current_dataset_variety)
            evaluations["Fold"].append(i)
            evaluations["Train_Test"].append("Train")
            evaluations["Accuracy"].append(acc)
            evaluations["Precision"].append(prec)
            evaluations["TPR"].append(tpr)
            evaluations["FPR"].append(fpr)
            evaluations["TPR_FPR_Thresholds"].append(thresholds)
            evaluations["AUC"].append(auc)
            evaluations["Sensitivity"].append(sens)
            evaluations["Specificity"].append(spec)
            evaluations["MCC"].append(mcc)

            ##################################################################################
            ##### Prediction and metrics for TEST dataset
            ##################################################################################

            y_seq, y_pred = model.predict(fold["X1_test"])
            label_pred = pred2label(y_pred)
            # Compute precision, recall, sensitivity, specifity, mcc
            acc = accuracy_score(fold["y_test"], label_pred)
            prec = precision_score(fold["y_test"],label_pred)

            conf = confusion_matrix(fold["y_test"], label_pred)
            if(conf[0][0]+conf[1][0]):
                sens = float(conf[0][0])/float(conf[0][0]+conf[1][0])
            else:
                sens = 0.0
            if(conf[1][1]+conf[0][1]):
                spec = float(conf[1][1])/float(conf[1][1]+conf[0][1])
            else:
                spec = 0.0
            if((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0])):
                mcc = (float(conf[0][0])*float(conf[1][1]) - float(conf[1][0])*float(conf[0][1]))/math.sqrt((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0]))
            else:
                mcc= 0.0
            fpr, tpr, thresholds = roc_curve(fold["y_test"], y_pred)
            auc = roc_auc_score(fold["y_test"], y_pred)

            evaluations["Model"].append(current_dataset_variety)
            evaluations["Kernel_Length"].append(None)
            evaluations["Dataset"].append(current_dataset_variety)
            evaluations["Fold"].append(i)
            evaluations["Train_Test"].append("Test")
            evaluations["Accuracy"].append(acc)
            evaluations["Precision"].append(prec)
            evaluations["TPR"].append(tpr)
            evaluations["FPR"].append(fpr)
            evaluations["TPR_FPR_Thresholds"].append(thresholds)
            evaluations["AUC"].append(auc)
            evaluations["Sensitivity"].append(sens)
            evaluations["Specificity"].append(spec)
            evaluations["MCC"].append(mcc)

            i = i+1
            del model
            tf.keras.backend.clear_session()

        ##################################################################################
        ##### Dump evaluations to a file
        ##################################################################################

#         evalPath = os.path.join(outPath, expName, "_Evaluation_All_Datasets")
#         if(not os.path.isdir(evalPath)):
#             os.makedirs(evalPath)

#         pickle.dump(evaluations,
#                     open(os.path.join(evalPath, "{}fold_evaluations.pickle".format(n_fold)), "wb"))



File: Data\Aziz\HS_990.csv
Positive: 495
Negative: 495

Train/Test model HS_990 on Fold #0.
Epoch 1/100
20/20 [==============================] - 0s 15ms/step - loss: 0.7529 - sequential_4_loss: 0.0753 - sequential_5_loss: 0.6776 - sequential_4_accuracy: 0.1760 - sequential_5_accuracy: 0.5908 - val_loss: 1.0935 - val_sequential_4_loss: 0.0731 - val_sequential_5_loss: 1.0203 - val_sequential_4_accuracy: 0.2965 - val_sequential_5_accuracy: 0.0377
Epoch 2/100
20/20 [==============================] - 0s 6ms/step - loss: 0.6875 - sequential_4_loss: 0.0700 - sequential_5_loss: 0.6175 - sequential_4_accuracy: 0.3371 - sequential_5_accuracy: 0.6445 - val_loss: 1.0912 - val_sequential_4_loss: 0.0658 - val_sequential_5_loss: 1.0253 - val_sequential_4_accuracy: 0.3645 - val_sequential_5_accuracy: 0.1195
Epoch 3/100
20/20 [==============================] - 0s 6ms/step - loss: 0.6551 - sequential_4_loss: 0.0629 - sequential_5_loss: 0.5921 - sequential_4_accuracy: 0.3765 - sequential_5_accuracy: 0.

Epoch 24/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5739 - sequential_4_loss: 0.0390 - sequential_5_loss: 0.5349 - sequential_4_accuracy: 0.6703 - sequential_5_accuracy: 0.7314 - val_loss: 1.2818 - val_sequential_4_loss: 0.0432 - val_sequential_5_loss: 1.2386 - val_sequential_4_accuracy: 0.6131 - val_sequential_5_accuracy: 0.3585
Epoch 25/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5667 - sequential_4_loss: 0.0383 - sequential_5_loss: 0.5285 - sequential_4_accuracy: 0.6774 - sequential_5_accuracy: 0.7472 - val_loss: 1.3021 - val_sequential_4_loss: 0.0428 - val_sequential_5_loss: 1.2593 - val_sequential_4_accuracy: 0.6167 - val_sequential_5_accuracy: 0.3585
Epoch 26/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5701 - sequential_4_loss: 0.0376 - sequential_5_loss: 0.5325 - sequential_4_accuracy: 0.6829 - sequential_5_accuracy: 0.7362 - val_loss: 1.2750 - val_sequential_4_loss: 0.0425 - val_sequential_5_loss: 1.2324 - va

20/20 [==============================] - 0s 7ms/step - loss: 0.5583 - sequential_4_loss: 0.0297 - sequential_5_loss: 0.5286 - sequential_4_accuracy: 0.7600 - sequential_5_accuracy: 0.7457 - val_loss: 1.1338 - val_sequential_4_loss: 0.0403 - val_sequential_5_loss: 1.0935 - val_sequential_4_accuracy: 0.6457 - val_sequential_5_accuracy: 0.4214
Epoch 48/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5599 - sequential_4_loss: 0.0294 - sequential_5_loss: 0.5305 - sequential_4_accuracy: 0.7633 - sequential_5_accuracy: 0.7551 - val_loss: 1.2361 - val_sequential_4_loss: 0.0403 - val_sequential_5_loss: 1.1958 - val_sequential_4_accuracy: 0.6481 - val_sequential_5_accuracy: 0.3774
Epoch 49/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5609 - sequential_4_loss: 0.0292 - sequential_5_loss: 0.5317 - sequential_4_accuracy: 0.7645 - sequential_5_accuracy: 0.7441 - val_loss: 1.2387 - val_sequential_4_loss: 0.0403 - val_sequential_5_loss: 1.1984 - val_sequential_

20/20 [==============================] - 0s 7ms/step - loss: 0.5531 - sequential_4_loss: 0.0256 - sequential_5_loss: 0.5275 - sequential_4_accuracy: 0.8028 - sequential_5_accuracy: 0.7409 - val_loss: 1.3048 - val_sequential_4_loss: 0.0412 - val_sequential_5_loss: 1.2636 - val_sequential_4_accuracy: 0.6490 - val_sequential_5_accuracy: 0.3648
Epoch 71/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5499 - sequential_4_loss: 0.0254 - sequential_5_loss: 0.5245 - sequential_4_accuracy: 0.8055 - sequential_5_accuracy: 0.7314 - val_loss: 1.4012 - val_sequential_4_loss: 0.0414 - val_sequential_5_loss: 1.3598 - val_sequential_4_accuracy: 0.6460 - val_sequential_5_accuracy: 0.3270
Epoch 72/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5496 - sequential_4_loss: 0.0253 - sequential_5_loss: 0.5243 - sequential_4_accuracy: 0.8074 - sequential_5_accuracy: 0.7425 - val_loss: 1.2245 - val_sequential_4_loss: 0.0413 - val_sequential_5_loss: 1.1832 - val_sequential_

20/20 [==============================] - 0s 7ms/step - loss: 0.5464 - sequential_4_loss: 0.0228 - sequential_5_loss: 0.5236 - sequential_4_accuracy: 0.8358 - sequential_5_accuracy: 0.7441 - val_loss: 1.3305 - val_sequential_4_loss: 0.0424 - val_sequential_5_loss: 1.2881 - val_sequential_4_accuracy: 0.6469 - val_sequential_5_accuracy: 0.3522
Epoch 94/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5528 - sequential_4_loss: 0.0228 - sequential_5_loss: 0.5300 - sequential_4_accuracy: 0.8347 - sequential_5_accuracy: 0.7378 - val_loss: 1.3955 - val_sequential_4_loss: 0.0427 - val_sequential_5_loss: 1.3528 - val_sequential_4_accuracy: 0.6508 - val_sequential_5_accuracy: 0.3208
Epoch 95/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5459 - sequential_4_loss: 0.0228 - sequential_5_loss: 0.5230 - sequential_4_accuracy: 0.8357 - sequential_5_accuracy: 0.7551 - val_loss: 1.2336 - val_sequential_4_loss: 0.0426 - val_sequential_5_loss: 1.1911 - val_sequential_

...

Epoch 4/100
13/13 [==============================] - 0s 8ms/step - loss: 0.5833 - sequential_1_loss: 0.0675 - sequential_2_loss: 0.5158 - sequential_1_accuracy: 0.3430 - sequential_2_accuracy: 0.7512 - val_loss: 1.0810 - val_sequential_1_loss: 0.0666 - val_sequential_2_loss: 1.0144 - val_sequential_1_accuracy: 0.3465 - val_sequential_2_accuracy: 0.3267
Epoch 5/100
13/13 [==============================] - 0s 8ms/step - loss: 0.5512 - sequential_1_loss: 0.0657 - sequential_2_loss: 0.4855 - sequential_1_accuracy: 0.3491 - sequential_2_accuracy: 0.7836 - val_loss: 1.2591 - val_sequential_1_loss: 0.0652 - val_sequential_2_loss: 1.1939 - val_sequential_1_accuracy: 0.3599 - val_sequential_2_accuracy: 0.2673
Epoch 6/100
13/13 [==============================] - 0s 8ms/step - loss: 0.5205 - sequential_1_loss: 0.0642 - sequential_2_loss: 0.4563 - sequential_1_accuracy: 0.3698 - sequential_2_accuracy: 0.7910 - val_loss: 1.1514 - val_sequential_1_loss: 0.0644 - val_sequential_2_loss: 1.0870 - val_s

13/13 [==============================] - 0s 8ms/step - loss: 0.3107 - sequential_1_loss: 0.0513 - sequential_2_loss: 0.2594 - sequential_1_accuracy: 0.5313 - sequential_2_accuracy: 0.8806 - val_loss: 2.1366 - val_sequential_1_loss: 0.0585 - val_sequential_2_loss: 2.0781 - val_sequential_1_accuracy: 0.4337 - val_sequential_2_accuracy: 0.4950
Epoch 28/100
13/13 [==============================] - 0s 8ms/step - loss: 0.3068 - sequential_1_loss: 0.0509 - sequential_2_loss: 0.2559 - sequential_1_accuracy: 0.5368 - sequential_2_accuracy: 0.9080 - val_loss: 2.4898 - val_sequential_1_loss: 0.0583 - val_sequential_2_loss: 2.4315 - val_sequential_1_accuracy: 0.4379 - val_sequential_2_accuracy: 0.4653
Epoch 29/100
13/13 [==============================] - 0s 7ms/step - loss: 0.3027 - sequential_1_loss: 0.0505 - sequential_2_loss: 0.2521 - sequential_1_accuracy: 0.5405 - sequential_2_accuracy: 0.9005 - val_loss: 2.3677 - val_sequential_1_loss: 0.0584 - val_sequential_2_loss: 2.3094 - val_sequential_

13/13 [==============================] - 0s 8ms/step - loss: 0.2625 - sequential_1_loss: 0.0454 - sequential_2_loss: 0.2171 - sequential_1_accuracy: 0.6051 - sequential_2_accuracy: 0.9179 - val_loss: 3.2637 - val_sequential_1_loss: 0.0581 - val_sequential_2_loss: 3.2056 - val_sequential_1_accuracy: 0.4570 - val_sequential_2_accuracy: 0.5248
Epoch 51/100
13/13 [==============================] - 0s 8ms/step - loss: 0.2747 - sequential_1_loss: 0.0452 - sequential_2_loss: 0.2295 - sequential_1_accuracy: 0.6076 - sequential_2_accuracy: 0.8955 - val_loss: 3.5764 - val_sequential_1_loss: 0.0581 - val_sequential_2_loss: 3.5183 - val_sequential_1_accuracy: 0.4526 - val_sequential_2_accuracy: 0.5248
Epoch 52/100
13/13 [==============================] - 0s 8ms/step - loss: 0.2596 - sequential_1_loss: 0.0450 - sequential_2_loss: 0.2146 - sequential_1_accuracy: 0.6086 - sequential_2_accuracy: 0.9030 - val_loss: 3.6019 - val_sequential_1_loss: 0.0581 - val_sequential_2_loss: 3.5438 - val_sequential_

13/13 [==============================] - 0s 8ms/step - loss: 0.2473 - sequential_1_loss: 0.0419 - sequential_2_loss: 0.2054 - sequential_1_accuracy: 0.6448 - sequential_2_accuracy: 0.9104 - val_loss: 4.6311 - val_sequential_1_loss: 0.0588 - val_sequential_2_loss: 4.5723 - val_sequential_1_accuracy: 0.4650 - val_sequential_2_accuracy: 0.5149
Epoch 74/100
13/13 [==============================] - 0s 8ms/step - loss: 0.2478 - sequential_1_loss: 0.0418 - sequential_2_loss: 0.2060 - sequential_1_accuracy: 0.6455 - sequential_2_accuracy: 0.9179 - val_loss: 4.5290 - val_sequential_1_loss: 0.0589 - val_sequential_2_loss: 4.4701 - val_sequential_1_accuracy: 0.4663 - val_sequential_2_accuracy: 0.5248
Epoch 75/100
13/13 [==============================] - 0s 8ms/step - loss: 0.2253 - sequential_1_loss: 0.0416 - sequential_2_loss: 0.1836 - sequential_1_accuracy: 0.6495 - sequential_2_accuracy: 0.9229 - val_loss: 4.6781 - val_sequential_1_loss: 0.0589 - val_sequential_2_loss: 4.6192 - val_sequential_

13/13 [==============================] - 0s 8ms/step - loss: 0.2106 - sequential_1_loss: 0.0392 - sequential_2_loss: 0.1714 - sequential_1_accuracy: 0.6767 - sequential_2_accuracy: 0.9403 - val_loss: 6.2001 - val_sequential_1_loss: 0.0600 - val_sequential_2_loss: 6.1401 - val_sequential_1_accuracy: 0.4599 - val_sequential_2_accuracy: 0.5149
Epoch 97/100
13/13 [==============================] - 0s 8ms/step - loss: 0.1875 - sequential_1_loss: 0.0391 - sequential_2_loss: 0.1483 - sequential_1_accuracy: 0.6781 - sequential_2_accuracy: 0.9478 - val_loss: 6.1601 - val_sequential_1_loss: 0.0600 - val_sequential_2_loss: 6.1001 - val_sequential_1_accuracy: 0.4602 - val_sequential_2_accuracy: 0.5050
Epoch 98/100
13/13 [==============================] - 0s 7ms/step - loss: 0.1995 - sequential_1_loss: 0.0390 - sequential_2_loss: 0.1605 - sequential_1_accuracy: 0.6786 - sequential_2_accuracy: 0.9527 - val_loss: 6.4184 - val_sequential_1_loss: 0.0601 - val_sequential_2_loss: 6.3583 - val_sequential_

In [10]:
# np.eye(12)[np.argmax(y_seq, axis = -1)][0]

## Visualization of Evaluation

In [11]:
# ##################################################################################
# ##### Add import statement here, to make this next part of code standalone executable
# ##################################################################################

# import os
# import pickle
# import matplotlib as mpl
# import matplotlib.pyplot as plt
# from matplotlib.ticker import ScalarFormatter, FormatStrFormatter
# import numpy as np
# import pandas as pd


In [12]:
# ##################################################################################
# ##### Load file and convert to dataframe for easy manipulation
# ##################################################################################

# evalPath = os.path.join(outPath, expName, "_Evaluation_All_Datasets")
# if(not os.path.isdir(evalPath)):
#     os.makedirs(evalPath)

# evaluations = pickle.load(open(os.path.join(evalPath, "{}fold_evaluations.pickle".format(n_fold)), "rb"))

In [13]:
# evaluations["Model"] = evaluations["Model"][0:20]
# evaluations_df = pd.DataFrame.from_dict(evaluations)

In [14]:
evaluations_df = pd.DataFrame.from_dict(evaluations)

##################################################################################
##### Group dataset (mean of metrics) by [Dataset, Model, Train_Test] combinations
##################################################################################

evaluations_df_grouped = evaluations_df.groupby(["Dataset", 
                                                 "Model", 
                                                 "Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

# DLNN_3 = evaluations_df_grouped[np.in1d(evaluations_df_grouped.index.get_level_values(1), ['DLNN_3'])]
# DLNN_5 = evaluations_df_grouped[np.in1d(evaluations_df_grouped.index.get_level_values(1), ['DLNN_5'])]

# DLNN_3_Train = DLNN_3[np.in1d(DLNN_3.index.get_level_values(2), ['Train'])]
# DLNN_3_Test = DLNN_3[np.in1d(DLNN_3.index.get_level_values(2), ['Test'])]

# DLNN_5_Train = DLNN_5[np.in1d(DLNN_5.index.get_level_values(2), ['Train'])]
# DLNN_5_Test = DLNN_5[np.in1d(DLNN_5.index.get_level_values(2), ['Test'])]

In [15]:
evaluations_df_grouped

Accuracy  Precision       AUC  Sensitivity  \
Dataset Model  Train_Test                                               
HS_990  HS_990 Test        0.559596   0.545633  0.579227     0.588780   
               Train       0.686111   0.638168  0.745951     0.785565   
MM_944  MM_944 Test        0.608100   0.583317  0.665111     0.657238   
               Train       0.725374   0.675852  0.810639     0.815681   
SN_628  SN_628 Test        0.517435   0.516819  0.541235     0.518118   
               Train       0.833587   0.767241  0.860100     0.954036   

                           Specificity       MCC  
Dataset Model  Train_Test                         
HS_990  HS_990 Test           0.545633  0.126497  
               Train          0.638168  0.397125  
MM_944  MM_944 Test           0.583317  0.227989  
               Train          0.675852  0.470664  
SN_628  SN_628 Test           0.516819  0.035009  
               Train          0.767241  0.693443

In [16]:
# ##################################################################################
# ##### Decide on metric to visualize
# ##################################################################################

# print("Metrics Available : ")
# print(list(evaluations_df_grouped.columns))

#### Select a metric to plot below:

In [17]:
# metric_to_plot = "Accuracy"

In [18]:
# ##################################################################################
# ##### Visualize with a multiple Bar chart
# ##################################################################################

# x = np.arange(len(DLNN_3_Train[metric_to_plot]))
# width = 0.15

# fig, ax = plt.subplots(figsize=(17,6))
# rects1 = ax.bar(x - (4*(width/2)), round(DLNN_3_Train[metric_to_plot]*100, 3), width, label='DLNN_3, Train')
# rects2 = ax.bar(x - (1.5*(width/2)), round(DLNN_5_Train[metric_to_plot]*100, 3), width, label='DLNN_5, Train')
# rects3 = ax.bar(x + (1.5*(width/2)), round(DLNN_3_Test[metric_to_plot]*100, 3), width, label='DLNN_3, Test')
# rects4 = ax.bar(x + (4*(width/2)), round(DLNN_5_Test[metric_to_plot]*100, 3), width, label='DLNN_5, Test')

# ## Custom y-axis tick labels
# ax.set_ylabel(metric_to_plot)
# ax.set_ylim([(math.floor(min(evaluations_df_grouped[metric_to_plot])*10)-1)*10, 
#             (math.ceil(max(evaluations_df_grouped[metric_to_plot])*10)+1)*10])
# # ax.set_ylim([80, 105])

# ## Custom x-axis tick labels
# ax.set_xticks(x)
# # ax.set_xticklabels(DLNN_3_Train.index.get_level_values(0))
# # ax.set_xticklabels([m+" - "+str(n) for m,n in 
# #                         zip(DLNN_3_Train.index.get_level_values(0),DLNN_3_Train.index.get_level_values(1))],
# #                   rotation=30)
# ax.set_xticklabels(DLNN_3_Train.index.get_level_values(0))

# ax.set_title(metric_to_plot+' by Dataset, Model, Train/Test')
# ax.legend(loc='upper left')

# def autolabel(rects):
#     for rect in rects:
#         height = rect.get_height()
#         ax.annotate('{}'.format(height),
#                     xy=(rect.get_x() + rect.get_width() / 2, height),
#                     xytext=(0, 3),  # 3 points vertical offset
#                     textcoords="offset points", 
#                     ha='center', va='bottom', rotation=90)

# autolabel(rects1)
# autolabel(rects2)
# autolabel(rects3)
# autolabel(rects4)

# plt.show()

### Store all metrics' plots to file

In [19]:
# ##################################################################################
# ##### Iteratively generate comparison plot using every metric
# ##################################################################################

# for metric_to_plot in list(evaluations_df_grouped.columns):
    
#     x = np.arange(len(DLNN_3_Train[metric_to_plot]))
#     width = 0.15

#     fig, ax = plt.subplots(figsize=(17,6))
#     rects1 = ax.bar(x - (4*(width/2)), round(DLNN_3_Train[metric_to_plot]*100, 3), width, label='DLNN_3, Train')
#     rects2 = ax.bar(x - (1.5*(width/2)), round(DLNN_5_Train[metric_to_plot]*100, 3), width, label='DLNN_5, Train')
#     rects3 = ax.bar(x + (1.5*(width/2)), round(DLNN_3_Test[metric_to_plot]*100, 3), width, label='DLNN_3, Test')
#     rects4 = ax.bar(x + (4*(width/2)), round(DLNN_5_Test[metric_to_plot]*100, 3), width, label='DLNN_5, Test')

#     ## Custom y-axis tick labels
#     ax.set_ylabel(metric_to_plot)
#     ax.set_ylim([(math.floor(min(evaluations_df_grouped[metric_to_plot])*10)-1)*10, 
#                 (math.ceil(max(evaluations_df_grouped[metric_to_plot])*10)+1)*10])
#     # ax.set_ylim([80, 105])

#     ## Custom x-axis tick labels
#     ax.set_xticks(x)
#     # ax.set_xticklabels(DLNN_3_Train.index.get_level_values(0))
#     # ax.set_xticklabels([m+" - "+str(n) for m,n in 
#     #                         zip(DLNN_3_Train.index.get_level_values(0),DLNN_3_Train.index.get_level_values(1))],
#     #                   rotation=30)
#     ax.set_xticklabels(DLNN_3_Train.index.get_level_values(0))

#     ax.set_title(metric_to_plot+' by Dataset, Model, Train/Test')
#     ax.legend(loc='upper left')

#     def autolabel(rects):
#         for rect in rects:
#             height = rect.get_height()
#             ax.annotate('{}'.format(height),
#                         xy=(rect.get_x() + rect.get_width() / 2, height),
#                         xytext=(0, 3),  # 3 points vertical offset
#                         textcoords="offset points", 
#                         ha='center', va='bottom', rotation=90)

#     autolabel(rects1)
#     autolabel(rects2)
#     autolabel(rects3)
#     autolabel(rects4)
    
#     plt.savefig(os.path.join(evalPath, "{}_DLNN_Comparison".format(metric_to_plot)))
#     plt.close()
    